In [17]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import SGD

In [18]:
from ROOT import TMVA, TFile, TTree, TCut
from subprocess import call
from os.path import isfile

In [19]:

# Setup TMVA
#TMVA.Tools.Instance()
TMVA.Tools.Instance()
#TMVA.PyMethodBase.PyInitialize()
output = TFile.Open('TMVA.root', 'RECREATE')
factory = TMVA.Factory('TMVARegression', output,
        '!V:!Silent:Color:DrawProgressBar:Transformations=D,G:AnalysisType=Regression')
# Load data
if not isfile('tmva_reg_example.root'):
    call(['curl', '-O', 'http://root.cern.ch/files/tmva_reg_example.root'])
data = TFile.Open('tmva_reg_example.root')
tree = data.Get('TreeR')
dataloader = TMVA.DataLoader('dataset')
for branch in tree.GetListOfBranches():
    name = branch.GetName()
    if name != 'fvalue':
        dataloader.AddVariable(name)
dataloader.AddTarget('fvalue')
dataloader.AddRegressionTree(tree, 1.0)
dataloader.PrepareTrainingAndTestTree(TCut(''),
        'nTrain_Regression=4000:SplitMode=Random:NormMode=NumEvents:!V')
# Generate model
# Define model
model = Sequential()
#model.add(Dense(64, activation='tanh', W_regularizer=l2(1e-5), input_dim=2))
model.add(Dense(64, activation='tanh', input_dim=2))
model.add(Dense(1, activation='linear'))
# Set loss and optimizer
model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01))
# Store model to file
model.save('model.h5')
model.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 257
Trainable params: 257
Non-trainable params: 0
_________________________________________________________________
DataSetInfo              : [dataset] : Added class "Regression"
                         : Add Tree TreeR of type Regression with 10000 events
                         : Dataset[dataset] : Class index : 0  name : Regression


In [24]:
# Book methods

factory.BookMethod(dataloader, TMVA.Types.kPyKeras, 'PyKeras', 'H:!V:VarTransform=D,G:FilenameModel=model.h5:NumEpochs=20:BatchSize=32', TMVA.Types.kPyKeras, 'H:!V:VarTransform=D,G:FilenameModel=model.h5:NumEpochs=20:BatchSize=32')
# Run TMVA
factory.TrainAllMethods()
factory.TestAllMethods()
factory.EvaluateAllMethods()


Factory                  : Train all methods
                         : ...nothing found to train
Factory                  : Test all methods
                         : ...nothing found to test
Factory                  : Evaluate all methods
                         : ...nothing found to evaluate


In [14]:
from ROOT import TMVA, TFile, TString
from array import array
from subprocess import call
from os.path import isfile

In [16]:
# Setup TMVA
TMVA.Tools.Instance()
#TMVA.PyMethodBase.PyInitialize()
reader = TMVA.Reader("Color:!Silent")
# Load data
if not isfile('tmva_reg_example.root'):
    call(['curl', '-O', 'http://root.cern.ch/files/tmva_reg_example.root'])
data = TFile.Open('tmva_reg_example.root')
tree = data.Get('TreeR')
branches = {}
for branch in tree.GetListOfBranches():
    branchName = branch.GetName()
    branches[branchName] = array('f', [-999])
    tree.SetBranchAddress(branchName, branches[branchName])
    if branchName != 'fvalue':
        reader.AddVariable(branchName, branches[branchName])
# Book methods
reader.BookMVA('TMVA.Types.kPyKeras', TString('dataset/weights/TMVARegression_PyKeras.weights.xml'))
# Print some example regressions
print('Some example regressions:')
for i in range(20):
    tree.GetEntry(i)
    print('True/MVA value: {}/{}'.format(branches['fvalue'][0],reader.EvaluateMVA('PyKeras')))

TypeError: none of the 2 overloaded methods succeeded. Full details:
  TMVA::IMethod* TMVA::Reader::BookMVA(const TString& methodTag, const TString& weightfile) =>
    FATAL error (C++ exception of type runtime_error)
  TMVA::IMethod* TMVA::Reader::BookMVA(TMVA::Types::EMVA methodType, const char* xmlstr) =>
    could not convert argument 1 (an integer is required)

<FATAL>                         : <BookMVA> fatal error: unable to open input weight file: dataset/weights/TMVARegression_PyKeras.weights.xml
***> abort program execution
